Let's step through the program and understand what it does:

In [10]:
from ipywidgets import interact_manual
from IPython.display import HTML, display

In [11]:
def load_input(fname):
    lines = [line.strip() for line in open(fname).readlines()]
    return lines

In [15]:
helpstr = {'addr': '(add register) stores into register {C} the result of adding register {A} and register {B}.',
          'addi': '(add immediate) stores into register {C} the result of adding register {A} and value {B}.',
          'seti': '(set immediate) stores value {A} into register {C}. (Input B is ignored.)',
          'mulr': '(multiply register) stores into register {C} the result of multiplying register {A} and register {B}.',
          'eqrr': '(equal register/register) sets register {C} to 1 if register {A} is equal to register {B}.\nOtherwise, register {C} is set to 0.',
          'gtrr': '(greater-than register/register) sets register {C} to 1 if register {A} is greater than register {B}. Otherwise, register {C} is set to 0.',
          'muli': '(multiply immediate) stores into register {C} the result of multiplying register {A} and value {B}.',
          'setr': '(set register) copies the contents of register {A} into register {C}. (Input B is ignored.)',
          'bani': '(bitwise AND immediate) stores into register {C} the result of the bitwise AND of register {A} and value {B}.'}

def annotate(parsed_program, ip_value, registers):
    
    instr = parsed_program[ip_value]
    instr_type, A, B, C = instr.split()
    A, B, C = int(A), int(B), int(C)
    
    parsed_copy = parsed_program[:]
    parsed_copy[ip_value] = "<span style='background-color: #FFFF00'>" + parsed_copy[ip_value] + "</span>"
    listing = "<div>" + "".join(["<p><i>{}: </i>".format(ind) + item + "</p>" for ind, item in enumerate(parsed_copy)]) + "</div>"
    
    register_html = '<p>' + ", ".join("<i>r{}: </i>{}".format(ind, item) for ind, item in enumerate(registers)) + '</p><p>' + helpstr[instr_type].format(A=A, B=B, C=C)
    
    style = """<style>
    p {
   line-height:20%;
}
    .mycontainer {
    width: 80%;
    margin: auto;
    font-family: monospace;
    font-size: 12px;
    
}
.myone {
    width: 20%;
    float: left;
}
.mytwo {
    margin-left: 20%;
    width: 80%;
}
.mytwo p {
    line-height:100%;
}
    </style>
    """
    container = """<section class="mycontainer">
    <div class="myone">{}</div>
    <div class="mytwo">{}</div>
    </section>""".format(listing, register_html)
    return HTML(style + container)

In [16]:
program = load_input('input21')
registers = [0, 0, 0, 0, 0, 0]
program_len = len([line for line in program if not line.startswith('#')])
parsed_program = [line for line in program if not line.startswith('#')]
# bind instrructions point to a register
instr = program[0]
ip_register = int(instr.split('#ip ')[1])
ip_value = registers[ip_register] 
count = 0

@interact_manual
def step(n=[1, 10]):
    global ip_value, count, registers
    for _ in range(n):
        if ip_value >= program_len or ip_value < 0:
            print('program finished')
        display(annotate(parsed_program, ip_value, registers))
        instr = parsed_program[ip_value]
        instr_type, A, B, C = instr.split()
        A, B, C = int(A), int(B), int(C)
        if instr_type == 'seti':
            registers[ip_register] = ip_value
            registers[C] = A
            ip_value = registers[ip_register]
            ip_value += 1
        elif instr_type == 'addi':
            registers[ip_register] = ip_value
            registers[C] = registers[A] + B
            ip_value = registers[ip_register]
            ip_value += 1
        elif instr_type == 'setr':
            registers[ip_register] = ip_value
            registers[C] = registers[A]
            ip_value = registers[ip_register]
            ip_value += 1
        elif instr_type == 'mulr':
            registers[ip_register] = ip_value
            registers[C] = registers[A] * registers[B]
            ip_value = registers[ip_register]
            ip_value += 1
        elif instr_type == 'muli':
            registers[ip_register] = ip_value
            registers[C] = registers[A] * B
            ip_value = registers[ip_register]
            ip_value += 1
        elif instr_type == 'addr':
            registers[ip_register] = ip_value
            registers[C] = registers[A] + registers[B]
            ip_value = registers[ip_register]
            ip_value += 1
        elif instr_type == 'eqrr':
            registers[ip_register] = ip_value
            if registers[A] == registers[B]:
                registers[C] = 1
            else:
                registers[C] = 0
            ip_value = registers[ip_register]
            ip_value += 1
        elif instr_type == 'gtrr':
            registers[ip_register] = ip_value
            if registers[A] > registers[B]:
                registers[C] = 1
            else:
                registers[C] = 0
            ip_value = registers[ip_register]
            ip_value += 1
        count += 1
    

interactive(children=(Dropdown(description='n', options=(1, 10), value=1), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))